# Instalando o Spark

In [1]:
# !pip install pyspark #==3.3.1

In [3]:
""" !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip """

' !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip ngrok-stable-linux-amd64.zip '

# Iniciando Sessão Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = (
    SparkSession.builder                  # Método da classe que constrói a sessão spark
      .appName("Exercicios Spark")  # Nome do App Spark
      .getOrCreate())                     # Verifica se há uma sessão ativa, e se não há, cria uma nova sessão

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

caminho_csv = "./base_de_dados.csv"

spark.read.csv(
      path=caminho_csv,
      header=True,
      sep=";",
      schema=schema_base_pix,
      timestampFormat="dd/MM/yyyy HH:mm"
).createOrReplaceTempView('base_pix')

# Exercícios
1. Descubra a quantidade total de transações na base de dados.
2. Qual foi a quantia total de dinheiro transacional em cada ano?
2. Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.
2. Retorne a quantidade de transações agrupadas por tipo de chave.
3. Descubra qual é a média, mínima e máxima do valor de todas as transações.
4. Quais foram as 5 maiores transações durante todo o período?
5. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações. 
6. Descubra qual foi a maior transação de cada mês na base de dados.
7. Crie uma nova coluna que contenha o valor da transação em dólar. 

In [ ]:
spark.sql(
    """
      select * from base_pix limit 10
    """
).show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|2022-04-08 01:47:00|
|  3|   57.58|    Arthur Gonca

In [ ]:
# Exercicio 1
# Descubra a quantidade total de transações na base de dados
spark.sql(
    """
      select count(*) as qtd_transacoes from base_pix
    """
).show()

+--------------+
|qtd_transacoes|
+--------------+
|           100|
+--------------+



In [ ]:
# Exercicio 2
# Qual foi a quantia total de dinheiro transacional em cada ano?

spark.sql(
    """
      select 
        year(data_transacao),
        sum(valor) 
      from base_pix 
      group by 1
    """
).show()

+--------------------+------------------+
|year(data_transacao)|        sum(valor)|
+--------------------+------------------+
|                2022|517980.58999999973|
|                2021| 848320.6000000001|
+--------------------+------------------+



In [ ]:
# Exercicio 3
# Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.

spark.sql(
    """
      select sum(valor)
      from base_pix
      where parte_creditada_banco = 'Nubank'
    """
).show()

+----------+
|sum(valor)|
+----------+
| 302224.64|
+----------+



In [ ]:
# Exercicio 4
# Retorne a quantidade de transações agrupadas por tipo de chave.

spark.sql(
    """
      select chave_pix_tipo, count(*)
      from base_pix
      group by chave_pix_tipo
    """
).show()

+--------------+--------+
|chave_pix_tipo|count(1)|
+--------------+--------+
|       celular|      22|
|         email|      29|
|           cpf|      49|
+--------------+--------+



In [ ]:
# Exercicio 5
# Descubra qual é a média, mínima e máxima do valor de todas as transações.

spark.sql(
    """
      select max(valor), min(valor), mean(valor) from base_pix limit 10
    """
).show()

+----------+----------+------------------+
|max(valor)|min(valor)|       mean(valor)|
+----------+----------+------------------+
|  95977.62|      0.03|13663.011899999998|
+----------+----------+------------------+



In [ ]:
# Exercicio 6
# Quais foram as 5 maiores transações durante todo o período?

spark.sql(
    """
      select * from base_pix 
      order by valor desc
      limit 5
    """
).show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|     chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| 54|95977.62|   Henrique Ferreira|             6269315|              Nubank|Sra. Maria Luiza ...|             68542779|                  BTG|           cpf|         14957860211|2021-09-22 10:17:00|
| 86|94736.79|Joao Vitor Cavalc...|            52265189|                 BTG|        Stella Gomes|             93858790|               Nubank|         email|stella.gomes@yaho...|2021-01-26 00:04:00|
| 97|

In [ ]:
# Exercicio 7
# Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações.
spark.sql(
    """
      select chave_pix_tipo, count(*)
      from base_pix
      where parte_debitada_banco = 'BTG' or parte_creditada_banco = 'BTG'
      group by 1
      order by 2 desc
    """
).show()

+--------------+--------+
|chave_pix_tipo|count(1)|
+--------------+--------+
|           cpf|      22|
|         email|      15|
|       celular|      13|
+--------------+--------+



In [ ]:
# Exercicio 8
# Descubra qual foi a maior transação de cada mês na base de dados.
spark.sql(
    """
    with cte_base_row(
        select 
        id, 
        valor, 
        date_format(data_transacao, 'y-MM') as ano_mes,
        row_number() over (
          partition by date_format(data_transacao, 'y-MM')
          order by valor desc
        ) as row_number
      from base_pix
    ) select
        id,
        valor,
        ano_mes
      from cte_base_row
      where row_number = 1
    """
).show()

+---+--------+-------+
| id|   valor|ano_mes|
+---+--------+-------+
| 86|94736.79|2021-01|
| 78|81977.98|2021-02|
| 88|78347.58|2021-03|
| 44|   459.2|2021-04|
| 68|  611.53|2021-05|
| 52|60139.23|2021-06|
| 76| 9320.87|2021-07|
| 73|  570.52|2021-08|
| 54|95977.62|2021-09|
| 32| 1508.83|2021-10|
| 71|80083.34|2021-11|
| 40|  829.87|2021-12|
| 79| 78559.4|2022-01|
| 35| 7944.02|2022-02|
| 55|35409.61|2022-03|
| 31|20875.64|2022-04|
| 47|38219.08|2022-05|
| 36|48714.95|2022-06|
| 18|49836.72|2022-07|
| 12| 4374.56|2022-08|
+---+--------+-------+
only showing top 20 rows



In [ ]:
# Exercicio 9
# Crie uma nova coluna que contenha o valor da transação em dólar.
spark.sql(
    """
      select id, valor, round(valor * 5.09, 2) as valor_dolar from base_pix limit 10
    """
).show()


+---+--------+-----------+
| id|   valor|valor_dolar|
+---+--------+-----------+
|  1|    9.93|      50.54|
|  2|   15.38|      78.28|
|  3|   57.58|     293.08|
|  4|53705.13|  273359.11|
|  5|25299.69|  128775.42|
|  6| 7165.06|   36470.16|
|  7|    6.16|      31.35|
|  8|  136.36|     694.07|
|  9|  574.39|    2923.65|
| 10|   42.88|     218.26|
+---+--------+-----------+

